In [1]:
# libraries

from datetime import datetime
import os
import glob
import requests 
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Web scrapping

In [2]:
# download data 
# =============

link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSc_2y5N0I67wDU38DjDh35IZSIS30rQf7_NYZhtYYGU1jJYT6_kDx4YpF-qw0LSlGsBYP8pqM_a1Pd/pubhtml#'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

# print(tbody)
# print(body)

In [4]:
head_row = [i.text for i in body[0].find_all('td')]
head_row

['Patient Number',
 'State Patient Number',
 'Date Announced',
 'Age Bracket',
 'Gender',
 'Detected City',
 'Detected District',
 'Detected State',
 'State code',
 'Current Status',
 'Notes',
 'Contracted from which Patient (Suspected)',
 'Nationality',
 'Type of transmission',
 'Status Change Date',
 'Source_1',
 'Source_2',
 'Source_3',
 'Backup Notes',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [5]:
contents = []

for i in range(len(body)):
    contents.append([i.text for i in body[i].find_all('td')])

# Saving to Dataframe

In [6]:
p_df = pd.DataFrame(contents[2:len(contents)], columns=head_row)
p_df.head()

,Patient Number,State Patient Number,Date Announced,Age Bracket,Gender,Detected City,Detected District,Detected State,State code,Current Status,...,Source_2,Source_3,Backup Notes,,,,,,,
0,1,KL-TS-P1,30/01/2020,20,F,Thrissur,Thrissur,Kerala,KL,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,,None
1,2,KL-AL-P1,02/02/2020,,,Alappuzha,Alappuzha,Kerala,KL,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,,Student from Wuhan,,,,,,,
2,3,KL-KS-P1,03/02/2020,,,Kasaragod,Kasaragod,Kerala,KL,Recovered,...,https://twitter.com/ANI/status/122422148580539...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,
3,4,DL-P1,02/03/2020,45,M,East Delhi (Mayur Vihar),East Delhi,Delhi,DL,Recovered,...,https://economictimes.indiatimes.com/news/poli...,,Travel history to Italy and Austria,,,,,,,
4,5,TS-P1,02/03/2020,24,M,Hyderabad,Hyderabad,Telangana,TG,Recovered,...,https://www.indiatoday.in/india/story/coronavi...,https://www.thehindu.com/news/national/coronav...,"Travel history to Dubai, Singapore contact",,,,,,,


# Data cleaning and transformations

In [7]:
# shape of dataframe
# ==================

p_df.shape

(6100, 26)

In [8]:
# columns
# =======

p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'State code', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
       'Source_3', 'Backup Notes', '', '', '', '', '', '', ''],
      dtype='object')

In [9]:
# selecting important columns only
# ================================

p_df = p_df.loc[:, :'Backup Notes']

In [10]:
# looking for missing values 
# ==========================

p_df.isna().sum()

Patient Number                               0
State Patient Number                         0
Date Announced                               0
Age Bracket                                  0
Gender                                       0
Detected City                                0
Detected District                            0
Detected State                               0
State code                                   0
Current Status                               0
Notes                                        0
Contracted from which Patient (Suspected)    0
Nationality                                  0
Type of transmission                         0
Status Change Date                           0
Source_1                                     0
Source_2                                     0
Source_3                                     0
Backup Notes                                 0
dtype: int64

In [11]:
# no. of empty strings in each column
# ===================================

print(p_df.shape)

for i in p_df.columns:
    print(i, '\t', p_df[p_df[i]==''].shape[0])

(6100, 19)
Patient Number 	 701
State Patient Number 	 4959
Date Announced 	 739
Age Bracket 	 5091
Gender 	 4652
Detected City 	 5224
Detected District 	 1680
Detected State 	 740
State code 	 740
Current Status 	 741
Notes 	 1958
Contracted from which Patient (Suspected) 	 4961
Nationality 	 5000
Type of transmission 	 3470
Status Change Date 	 850
Source_1 	 950
Source_2 	 4524
Source_3 	 5813
Backup Notes 	 5740


In [12]:
# replacing empty strings with np.nan
# ==================================-

print(p_df.shape)

p_df = p_df.replace(r'', np.nan, regex=True)
p_df.isna().sum()

(6100, 19)


Patient Number                                701
State Patient Number                         4959
Date Announced                                739
Age Bracket                                  5091
Gender                                       4652
Detected City                                5224
Detected District                            1680
Detected State                                740
State code                                    740
Current Status                                741
Notes                                        1958
Contracted from which Patient (Suspected)    4961
Nationality                                  5000
Type of transmission                         3470
Status Change Date                            850
Source_1                                      950
Source_2                                     4524
Source_3                                     5813
Backup Notes                                 5740
dtype: int64

In [13]:
# droping empty rows (row with just row number but without patient entry
# ======================================================================

p_df.dropna(subset=['Detected State'], inplace=True)
print(p_df.shape)
p_df.isna().sum()

(5360, 19)


Patient Number                                  0
State Patient Number                         4219
Date Announced                                  0
Age Bracket                                  4351
Gender                                       3912
Detected City                                4484
Detected District                             940
Detected State                                  0
State code                                      0
Current Status                                  1
Notes                                        1218
Contracted from which Patient (Suspected)    4221
Nationality                                  4260
Type of transmission                         2730
Status Change Date                            110
Source_1                                      210
Source_2                                     3784
Source_3                                     5073
Backup Notes                                 5000
dtype: int64

In [14]:
p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'State code', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
       'Source_3', 'Backup Notes'],
      dtype='object')

In [15]:
# rename dateframe columns 
# ========================

p_df.columns = ['_'.join(col.lower().split()) for col in p_df.columns]
p_df.rename(columns = {'contracted_from_which_patient_(suspected)':'suspected_contacted_patient'}, inplace=True)
p_df.sample(5)

,patient_number,state_patient_number,date_announced,age_bracket,gender,detected_city,detected_district,detected_state,state_code,current_status,notes,suspected_contacted_patient,nationality,type_of_transmission,status_change_date,source_1,source_2,source_3,backup_notes
1980,1981,AP-P93,01/04/2020,43,M,NaN,Guntur,Andhra Pradesh,AP,Hospitalized,Details awaited,NaN,NaN,NaN,01/04/2020,https://twitter.com/ArogyaAndhra/status/124538...,NaN,NaN,NaN
2912,2913,NaN,03/04/2020,NaN,NaN,NaN,NaN,Telangana,TG,Hospitalized,Details awaited,NaN,NaN,NaN,03/04/2020,https://twitter.com/Eatala_Rajender/status/124...,NaN,NaN,NaN
4693,4694,NaN,06/04/2020,NaN,NaN,NaN,Bhopal,Madhya Pradesh,MP,Hospitalized,Details awaited,NaN,NaN,NaN,06/04/2020,https://twitter.com/JansamparkMP/status/124718...,NaN,NaN,NaN
2025,2026,NaN,01/04/2020,NaN,NaN,NaN,NaN,Telangana,TG,Hospitalized,Details awaited,NaN,NaN,NaN,01/04/2020,https://twitter.com/Eatala_Rajender/status/124...,NaN,NaN,NaN
2059,2060,NaN,01/04/2020,NaN,NaN,NaN,Indore,Madhya Pradesh,MP,Hospitalized,Details awaited,NaN,NaN,NaN,01/04/2020,https://twitter.com/ANI/status/124542865606912...,NaN,NaN,NaN


In [16]:
# creating patient id column from patient number
# ===============================================

p_df['p_id'] = p_df['patient_number'].apply(lambda x : 'P'+str(x))
p_df.columns

Index(['patient_number', 'state_patient_number', 'date_announced',
       'age_bracket', 'gender', 'detected_city', 'detected_district',
       'detected_state', 'state_code', 'current_status', 'notes',
       'suspected_contacted_patient', 'nationality', 'type_of_transmission',
       'status_change_date', 'source_1', 'source_2', 'source_3',
       'backup_notes', 'p_id'],
      dtype='object')

In [17]:
p_df = p_df.loc[:, :'backup_notes']

# Saving Data

In [18]:
p_df.sample(5)

,patient_number,state_patient_number,date_announced,age_bracket,gender,detected_city,detected_district,detected_state,state_code,current_status,notes,suspected_contacted_patient,nationality,type_of_transmission,status_change_date,source_1,source_2,source_3,backup_notes
1710,1711,AP-P79,01/04/2020,35,M,NaN,Y.S.R.,Andhra Pradesh,AP,Hospitalized,Travelled to Delhi,E0,NaN,Local,01/04/2020,https://twitter.com/ArogyaAndhra/status/124522...,https://twitter.com/ANI/status/124525248096919...,NaN,NaN
3450,3451,TN-P483,04/04/2020,NaN,M,NaN,Ranipet,Tamil Nadu,TN,Hospitalized,Travelled to Delhi,E0,NaN,Local,04/04/2020,https://twitter.com/ANI/status/124641832005770...,NaN,NaN,NaN
2364,2365,NaN,02/04/2020,NaN,NaN,NaN,Ahmadnagar,Maharashtra,MH,Hospitalized,NaN,NaN,NaN,NaN,02/04/2020,https://twitter.com/ANI/status/124569351450080...,NaN,NaN,NaN
1981,1982,AP-P94,01/04/2020,NaN,M,NaN,Guntur,Andhra Pradesh,AP,Hospitalized,Details awaited,NaN,NaN,NaN,01/04/2020,https://twitter.com/ArogyaAndhra/status/124538...,NaN,NaN,NaN
2763,2764,TN-P330,03/04/2020,NaN,NaN,NaN,Chennai,Tamil Nadu,TN,Hospitalized,Travelled to Delhi,E0,NaN,Local,03/04/2020,https://twitter.com/ANI/status/124604385644717...,https://twitter.com/ANI/status/124605490629894...,NaN,NaN


In [19]:
p_df.to_csv('patients_data.csv', index=False)